# Installing & Imports

In [146]:
conda install pandas==0.24.2

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - pandas==0.24.2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         153 KB
    ------------------------------------------------------------
                                           Total:         153 KB

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2019.11.~ --> pkgs/main::ca-certificates-2020.1.1-0
  openssl            conda-forge::openssl-1.1.1d-h516909a_0 --> pkgs/main::openssl-1.1.1d-h7b6447c_4

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi                                       conda-forge --> pkgs/main



certifi-2019.11.28   | 153 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying

In [147]:
conda upgrade --all -y

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [148]:
conda install snorkel==0.9.0 -c conda-forge

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - snorkel==0.9.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         149 KB

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates     pkgs/main::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2019.11.28-hecc5488_0
  certifi                                         pkgs/main --> conda-forge
  openssl              pkgs/main::openssl-1.1.1d-h7b6447c_4 --> conda-forge::openssl-1.1.1d-h516909a_0



certifi-2019.11.28   | 149 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction

In [149]:
conda install -c conda-forge textblob

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [4]:
from snorkel.labeling import LFAnalysis
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier,LabelModel
from snorkel.preprocess import preprocessor
from textblob import TextBlob
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
from csv import writer
import re
import pickle
import time
import json




In [5]:
#Spark 

# Spark Environment
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
import pyspark

number_cores = 4
memory_gb = 16
conf = (
    pyspark.SparkConf()
        .setMaster('local[{}]'.format(number_cores))
        .set('spark.driver.memory', '{}g'.format(memory_gb))
)
sc = pyspark.SparkContext.getOrCreate(conf=conf)
print(sc)

# get the context
spark = pyspark.sql.SparkSession.builder.getOrCreate()
print(spark) 

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

ModuleNotFoundError: No module named 'pyspark'

# Web Scraping

## Browse all from DVD releases page

In [24]:
main = 'https://www.rottentomatoes.com/api/private/v2.0/browse?maxTomato=100&services=amazon%3Bhbo_go%3Bitunes%3Bnetflix_iw%3Bvudu%3Bamazon_prime%3Bfandango_now&certified&sortBy=release&type=dvd-streaming-all&page='

In [37]:
# Get movie url
movie_url = []
start_page = 1 ; end_page = 1
while start_page <= end_page:
#     time.sleep(7)
    url = main + str(start_page)
    response = requests.get(url)
    if response.status_code !=200:
        print('Request error')
        break
    file = json.loads(response.text)
    for i in file['results']:
        movie_url = movie_url + [i['url']]
    start_page +=1

In [38]:
print('Examples for the url:\n')
for i in range(3):
    print(movie_url[i])
print('\nNumber of movies in list: {}'.format(len(movie_url)))

Examples for the url:

/m/frozen_ii
/m/playmobil_the_movie
/m/a_shaun_the_sheep_movie_farmageddon

Number of movies in list: 32


In [39]:
# Split into lists of 50 movies to do the scraping
movie_url_split = [movie_url[i:i+50] for i in range(0,600,50)]

In [40]:
len(movie_url_split)

12

In [42]:
# Get reviews from the web
reviews = []
titles = []
ratings = []
for split in movie_url_split: # Loop through each split
#     time.sleep(7)
    for title in split: # Loop through each movie title
        url = 'https://www.rottentomatoes.com'+title
#         time.sleep(7)
        response = requests.get(url)
        # Check the request status code
        if response.status_code != 200:
            print('Request error')
            break
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Get labels from each review (fresh vs. rotten)
        fresh_rotten = soup.find_all(class_="review_quote")
        
        # Get movie title
        title = soup.find(class_="mop-ratings-wrap__title mop-ratings-wrap__title--top").getText()
        
        # Get reviews
        review = soup.find_all('blockquote')
        for i in review:
            j = str(i.contents[1])
            j = j.replace("<p>\n                    \n                        ","")
            j = j.replace("\n                    \n                </p>","")
            reviews = reviews + [j]
            titles = titles + [title]
        
        # Identify labels
        for i in fresh_rotten:
            temp = str(i.findChildren()[2])
            if re.search('rotten',temp):
                ratings = ratings + ['rotten']
            else:
                ratings = ratings + ['fresh']
            

In [43]:
# Create the DataFrame to store the scraped data
df = pd.DataFrame([titles,reviews,ratings],index = ['title','review','rating']).T

In [44]:
# Clean the data (drop duplicates, check missing values etc.)
df = df.drop_duplicates()
df = df.replace([None],np.nan)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 327 entries, 0 to 327
Data columns (total 3 columns):
title     327 non-null object
review    327 non-null object
rating    327 non-null object
dtypes: object(3)
memory usage: 10.2+ KB


In [45]:
df.dropna(inplace=True)
df.head(3)

,title,review,rating
0,Frozen II,A distressingly unnecessary (and fairly tediou...,rotten
1,Frozen II,Frozen II is the exact kind of sequel that mor...,fresh
2,Frozen II,[A]n ultimately satisfying story of more matur...,fresh


In [46]:
# Export to CSV files
# df.to_csv('web_scraping_rotten_tomatoes.csv')

# Reading & Preparing TSV file 

In [20]:
# Read TSV file
tsv_reviews = pd.read_csv('/project/reviews.tsv', sep='\t', header=0, encoding='unicode_escape')

In [21]:
tsv_reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [22]:
# Extract review and fresh columns
tsv_reviews = pd.DataFrame(tsv_reviews, columns = ['review', 'fresh'])

In [23]:
tsv_reviews.head()

,review,fresh
0,A distinctly gallows take on contemporary fina...,fresh
1,It's an allegory in search of a meaning that n...,rotten
2,... life lived in a bubble in financial dealin...,fresh
3,Continuing along a line introduced in last yea...,fresh
4,... a perverse twist on neorealism...,fresh


In [24]:
tsv_reviews.isnull().sum()

review    5563
fresh        0
dtype: int64

In [25]:
# drop NaN rows in reviews
index_name = tsv_reviews[(tsv_reviews['review'].isnull())].index
tsv_reviews.drop(index_name, inplace= True)

In [26]:
tsv_reviews.isnull().sum()

review    0
fresh     0
dtype: int64

In [27]:
# rename fresh as 1 and rotten as 0
tsv_reviews['fresh'].replace({'fresh':'1', 'rotten':'0'}, inplace = True)

In [28]:
#Rename columns
tsv_reviews.rename(columns={'fresh':'Freshness','review':'Review'},inplace=True)
tsv_reviews = tsv_reviews.sample(5000)
#take 5000
tsv_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 52464 to 33016
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Review     5000 non-null   object
 1   Freshness  5000 non-null   object
dtypes: object(2)
memory usage: 117.2+ KB


# Reading & Preparing CSV file 

In [29]:
# Read CSV file
csv_reviews= pd.read_csv('/project/rotten_tomatoes_reviews.csv')
csv_reviews.head()


,Freshness,Review
0,1,"Manakamana doesn't answer any questions, yet ..."
1,1,Wilfully offensive and powered by a chest-thu...
2,0,It would be difficult to imagine material mor...
3,0,Despite the gusto its star brings to the role...
4,0,If there was a good idea at the core of this ...


In [30]:
#Swap Freshness and Review 
columns_titles = ["Review","Freshness"]
csv_reviews=csv_reviews.reindex(columns=columns_titles)
csv_reviews = csv_reviews.sample(5000)

csv_reviews.head()
csv_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 118357 to 81277
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Review     5000 non-null   object
 1   Freshness  5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 117.2+ KB


# Web Scraping & Preparing scrapped data 

In [31]:
#Read Web Scraping Data
web_scraping_reviews= pd.read_csv('/project/web_scraping_rotten_tomatoes.csv')
web_scraping_reviews.head()

web_scraping_reviews = web_scraping_reviews.sample(5000)

In [32]:
# rename fresh as 1 and rotten as 0
web_scraping_reviews['rating'].replace({'fresh':'1', 'rotten':'0'}, inplace = True)

#Rename Rating to Review 
web_scraping_reviews.rename(columns={'rating':'Freshness', 'review':'Review'},inplace=True)
web_scraping_reviews.head()

# Extract Review and Freshness columns
web_scraping_reviews= pd.DataFrame(web_scraping_reviews, columns = ['Review', 'Freshness'])

In [33]:
web_scraping_reviews.head()

,Review,Freshness
2275,It's an uneven but affectionate portrait of tw...,1
591,If there is a meaningful difference between pe...,1
4462,Little is a really charming and funny movie......,1
5173,Happy Death 2U answers all of the first film's...,1
3563,If you're hoping El Chicano might do for the L...,1


# Combining all the data together 

In [34]:
# csv_reviews.info()
# tsv_reviews.info()
# web_scraping_reviews.info()



# Concat two files into all_reviews
all_reviews=pd.concat([csv_reviews, tsv_reviews,web_scraping_reviews],axis=0, sort=False)
all_reviews.head()

,Review,Freshness
118357,Director Hugh Hudson (Chariots of Fire) visit...,0
327911,Black Panther pounces toward the head of the ...,1
15135,[It's] the sort of intellectually offensive c...,0
433159,"""Suicide Squad"" has some fun and worthwhile m...",0
334432,'Grey' deserved a concentrated attack. It get...,0


In [35]:
all_reviews.shape

(15000, 2)

# Split into test and training set 

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
all_reviews['Freshness'] = all_reviews['Freshness'].astype(int)

In [38]:
train, test = train_test_split(all_reviews,test_size=0.2,stratify = all_reviews['Freshness'])

In [39]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 30798 to 3724
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Review     12000 non-null  object
 1   Freshness  12000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 281.2+ KB


In [40]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 53798 to 2824
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Review     3000 non-null   object
 1   Freshness  3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 70.3+ KB


In [41]:
train.head()

,Review,Freshness
30798,A film that simply doesn't have enough cinemat...,0
1010,The last twenty minutes are gratuitous violenc...,0
4638,I like this character and I like this movie. B...,1
2689,This crime procedural is consistently fascinat...,1
3781,While Dev Patel brings the best performance po...,0


In [42]:
#get rid off training labels 
train = train.drop('Freshness', 1)

In [43]:
test['Freshness'].value_counts()

1    1751
0    1249
Name: Freshness, dtype: int64

In [44]:
#From labelled test set, extract a sample to find out about which labelling functions could be written
#Not sure how big the development split_ can be --> take sample of 1000 data points 

development_split = test.sample(1000,random_state=42)
development_split.head()


,Review,Freshness
4088,"Pet Sematary is a mechanically sound, if not e...",0
1399,The latest in Nicolas Cage's seemingly endless...,0
16248,"One of the funniest, most joyous comedies to c...",1
15424,"... frustratingly humorless, harsh, and hollow...",1
5267,It trades less in narrative than it does symbo...,1


In [45]:
#For finding labelling functions: 
development_split

,Review,Freshness
4088,"Pet Sematary is a mechanically sound, if not e...",0
1399,The latest in Nicolas Cage's seemingly endless...,0
16248,"One of the funniest, most joyous comedies to c...",1
15424,"... frustratingly humorless, harsh, and hollow...",1
5267,It trades less in narrative than it does symbo...,1
...,...,...
3510,"Earnest, old-fashioned social-justice drama fo...",1
284244,There's some fun to be had.,1
98965,"While Caine is, without any doubt, absolutely...",0
1533,NETIZENS is an arresting cry for solidarity.,1


In [46]:
development_split.count()

Review       1000
Freshness    1000
dtype: int64

In [47]:
development_split.to_csv('development_split.csv')

In [48]:
development_split.count()

Review       1000
Freshness    1000
dtype: int64

In [49]:
development_split.head()

,Review,Freshness
4088,"Pet Sematary is a mechanically sound, if not e...",0
1399,The latest in Nicolas Cage's seemingly endless...,0
16248,"One of the funniest, most joyous comedies to c...",1
15424,"... frustratingly humorless, harsh, and hollow...",1
5267,It trades less in narrative than it does symbo...,1


In [50]:
#Might have to get rid off index?

development_split[development_split['Freshness'] !=1].count()

Review       414
Freshness    414
dtype: int64

In [51]:
development_split['Freshness'].value_counts()

1    586
0    414
Name: Freshness, dtype: int64